In [1]:
import cv2
import pandas as pd
import numpy as np

In [2]:
img = cv2.imread("./data/masks/0363100012175978_4.380233.png")
cv2.imshow("Display window", img)
k = cv2.waitKey(0) # Wait for a keystroke in the window

In [3]:
image_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# Convert the image to HSV for better color segmentation
hsv = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2HSV)

# save CSV to read the color values of the image 
hsv_2d = hsv.reshape((-1,3))
df = pd.DataFrame(hsv_2d)
tiles_csv_path = './data/junk/hsv.csv'
df.to_csv(tiles_csv_path, index=False, header=False)

'not integrated in the code, just read from the file'

# retrieved color values
sky_val = np.array([0,0,255])
cl_fac_val = np.array([60,185,160])
window_fac_val = np.array([102,211,180])
door_fac_val = np.array([14,241,255])

mask = cv2.inRange(hsv, window_fac_val, window_fac_val)

#save image
cv2.imwrite("data/cv2_window_mask.png", mask)

# show image 
    # cv2.imshow("img", mask)
    # cv2.waitKey(0)
    # cv.destroyAllWindows()

True

In [7]:
contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
# borders = []

# for contour in contours:
#     borders.append(cv2.boundingRect(contour))

# # # in order of X, Y,(top-left coordinate of the rectangle) and width, height 
# print(borders)

# Approximate contours to polygons + get bounding rects
contours_poly = [None]*len(contours)
boundRect = [None]*len(contours)

for i, c in enumerate(contours):
    contours_poly[i] = cv2.approxPolyDP(c, 3, True)
    boundRect[i] = cv2.boundingRect(contours_poly[i])
    # borders.append(boundRect[i])
# print(borders)
print(boundRect)

drawing = np.zeros((mask.shape[0], mask.shape[1], 3), dtype=np.uint8)
# Draw polygonal contour + bonding rects + circles
for i in range(len(contours)):
    color = (0,0,255)
    cv2.rectangle(drawing, (int(boundRect[i][0]), int(boundRect[i][1])), \
    (int(boundRect[i][0]+boundRect[i][2]), int(boundRect[i][1]+boundRect[i][3])), color, 2)

cv2.imshow('Contours', drawing)
k = cv2.waitKey(0) # Wait for a keystroke in the window
 

[(106, 738, 32, 73), (58, 735, 33, 77), (17, 733, 29, 28), (58, 661, 35, 58), (15, 660, 32, 58), (105, 658, 34, 62), (106, 588, 34, 48), (13, 586, 33, 48), (59, 584, 34, 52), (60, 526, 31, 42)]


In [8]:
#list heights of the upper and lower corner of each window
y_bounds = []
for border in boundRect:
    y_bound = []
    y_bound.append(border[1])
    lower_corner = border[1] + border[3]
    y_bound.append(lower_corner)
    y_bounds.append(y_bound)
print(y_bounds)

# TODO somehow??? the retrieved values seem not to match the visual output... solved!!

[[738, 811], [735, 812], [733, 761], [661, 719], [660, 718], [658, 720], [588, 636], [586, 634], [584, 636], [526, 568]]


In [6]:
# first cluster-tests     
print(y_bounds)
if y_bounds[1][1] <= y_bounds[0][0] and y_bounds[0][1] <= y_bounds[1][0]: 
    print("yay!")
else: 
    print("nay..")    

[[738, 811], [735, 812], [733, 761], [661, 719], [660, 718], [658, 720], [588, 636], [586, 634], [584, 636], [526, 568]]
nay..


In [9]:
# first cluster-tests     
print(y_bounds)
if y_bounds[1][0] <= y_bounds[0][1] and y_bounds[0][0] <= y_bounds[1][1]: 
    print("yay!")
else: 
    print("nay..")    

[[738, 811], [735, 812], [733, 761], [661, 719], [660, 718], [658, 720], [588, 636], [586, 634], [584, 636], [526, 568]]
yay!


In [25]:
window_clusters = []
for index, window_1 in enumerate(y_bounds): 
    window_clusters.append(index)
    cluster = []
    for window_2 in y_bounds: 
        if window_2[0] <= window_1[1] and window_1[0] <= window_2[1]: 
            # print("yay!")
            cluster.append(window_2)
        # else: 
            # print("nay..")
        # if window_1 is in cluster: 
    window_clusters.append(cluster)  

print(len(window_clusters))  
print(window_clusters)
print(y_bounds)

20
[0, [[738, 811], [735, 812], [733, 761]], 1, [[738, 811], [735, 812], [733, 761]], 2, [[738, 811], [735, 812], [733, 761]], 3, [[661, 719], [660, 718], [658, 720]], 4, [[661, 719], [660, 718], [658, 720]], 5, [[661, 719], [660, 718], [658, 720]], 6, [[588, 636], [586, 634], [584, 636]], 7, [[588, 636], [586, 634], [584, 636]], 8, [[588, 636], [586, 634], [584, 636]], 9, [[526, 568]]]
[[738, 811], [735, 812], [733, 761], [661, 719], [660, 718], [658, 720], [588, 636], [586, 634], [584, 636], [526, 568]]


In [37]:
indow_clusters = []
for index, window_1 in enumerate(y_bounds):
    print(window_1) 
    window_clusters.append(index)
    cluster = []
    for window_2 in y_bounds:
        low_bound = []
        if window_2[0] <= window_1[1]: 
            low_bound = window_2[0]
            print(low_bound)
        # elif window_1[0] <= window_2[1]:
            # high_bound = window_2[1]
            # print(high_bound)
        #         new_boundary.append(window_2[1])
        #     else: 
        #         new_boundary.append(window_1[0])
        #     print(new_boundary)       
            # print("yay!")
            # cluster.append(window_2)
        # else: 
            # print("nay..")
        # if window_1 is in cluster: 
    window_clusters.append(cluster)  

[738, 811]
738
735
733
661
660
658
588
586
584
526
[735, 812]
738
735
733
661
660
658
588
586
584
526
[733, 761]
738
735
733
661
660
658
588
586
584
526
[661, 719]
661
660
658
588
586
584
526
[660, 718]
661
660
658
588
586
584
526
[658, 720]
661
660
658
588
586
584
526
[588, 636]
588
586
584
526
[586, 634]
588
586
584
526
[584, 636]
588
586
584
526
[526, 568]
526


In [39]:
for index, window_1 in enumerate(y_bounds):
    print(window_1) 
    window_clusters.append(index)
    cluster = []
    for window_2 in y_bounds:
        high_bound = []
        print(window_1[0], window_2[1])
        if window_1[0] <= window_2[1]:
        
            high_bound = window_2[1]
            print(high_bound)

[738, 811]
738 811
811
738 812
812
738 761
761
738 719
738 718
738 720
738 636
738 634
738 636
738 568
[735, 812]
735 811
811
735 812
812
735 761
761
735 719
735 718
735 720
735 636
735 634
735 636
735 568
[733, 761]
733 811
811
733 812
812
733 761
761
733 719
733 718
733 720
733 636
733 634
733 636
733 568
[661, 719]
661 811
811
661 812
812
661 761
761
661 719
719
661 718
718
661 720
720
661 636
661 634
661 636
661 568
[660, 718]
660 811
811
660 812
812
660 761
761
660 719
719
660 718
718
660 720
720
660 636
660 634
660 636
660 568
[658, 720]
658 811
811
658 812
812
658 761
761
658 719
719
658 718
718
658 720
720
658 636
658 634
658 636
658 568
[588, 636]
588 811
811
588 812
812
588 761
761
588 719
719
588 718
718
588 720
720
588 636
636
588 634
634
588 636
636
588 568
[586, 634]
586 811
811
586 812
812
586 761
761
586 719
719
586 718
718
586 720
720
586 636
636
586 634
634
586 636
636
586 568
[584, 636]
584 811
811
584 812
812
584 761
761
584 719
719
584 718
718
584 720
720
584 636
6

In [ ]:
if window_2 is in window_clusters: 
    skip window_2
    

In [ ]:
# TODO
'next step, if the windows match (yay!):'
'1. change the original to the new extremes (see drawing notebook)'
'2. make list with the original window and the new window'
'3. loop over new situation: compare with new extremes and add window if windows match (yay!))'